In [102]:
#ONE TIME USE ONLY

from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import random
import csv
import shutil
import glob

src = 'C:/Users/Camille/Downloads/edited white' # Folder to copy images
parent = 'C:/Users/Camille/source/repos/CED-CRNN/raw/doctors/' #parent folder of images, split, and label
aug_dir = 'C:/Users/Camille/Downloads/split w no augment/train_augmented' #THIS SHOULD BE EMPTY

datagen = ImageDataGenerator(					
		width_shift_range= 25,
		height_shift_range= 10,
		rotation_range = 5,
		shear_range = 1,
		zoom_range = 0.2)

def augment_images(image_path, filename):
		img = load_img(image_path)
		x = img_to_array(img)
		x = x.reshape((1, ) + x.shape)

		i = 0
		for batch in datagen.flow(x, batch_size = 16, shuffle=True, seed=1,
								save_to_dir =aug_dir,
								save_prefix =filename+'_'+str(i), save_format ='png'):
			i += 1
			if i > 5:
				break

allFileNames = os.listdir(src)
random.seed(1)

def createCSV(csv_file, src):
	with open(csv_file, 'w', newline='') as file:
		writer = csv.writer(file)
		writer.writerow(["image", "label"])
		for name in src:
			if name.endswith('.png'):
				med = name.split("_")
				writer.writerow([name[:-4], med[1]])

createCSV('data.csv', allFileNames)
df = pd.read_csv('data.csv')

#split
train_set,test_set = train_test_split(df,stratify=df["label"],test_size=0.3)
train_set,val_set = train_test_split(train_set,stratify=train_set["label"],test_size=0.1)

#augment training images
for image in train_set['image']:
	image_path = os.path.join(src,image+'.png')
	augment_images(image_path, image)

train_aug = os.listdir(aug_dir)
createCSV('train_aug.csv', train_aug)
train_set = pd.read_csv('train_aug.csv')

split = ['train', 'val', 'test']
split2 = [train_set, val_set, test_set]

#create split txt files
for file, sets in zip(split,split2):
    with open(parent + 'split/' + file + '.txt', 'w', newline='') as f:
        for image in sets['image']:
            f.write(image + '\n')

#copy test and validation images
for i in range(1, len(split2)):
	for image in split2[i]['image']:
		shutil.copy(src+'/'+image+'.png', parent + 'images')

# copy augmented training images
for image in train_set['image']:
	shutil.copy(aug_dir+'/'+image+'.png',  parent + 'images')